In [1]:
import matplotlib.pyplot as plt

#Data Manipulation
import pandas as pd
import numpy as np

# Machine Learning
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import Input, Dense, Dropout
import tensorflow as tf

df = pd.read_csv("collected_data_180_step_1.csv")

df_shuffled = df.sample(frac=1, random_state=42)
df = df_shuffled
print("From local")

df.head()


# Train test split
X_train_df, X_test_df = train_test_split(df, test_size=0.2, random_state=42)

# Train values
y1_train = np.array(X_train_df["Theta1"])
y2_train = np.array(X_train_df["Theta2"])
y3_train = np.array(X_train_df["Theta3"])

# Test values
y1_test = np.array(X_test_df["Theta1"])
y2_test = np.array(X_test_df["Theta2"])
y3_test = np.array(X_test_df["Theta3"])

# Remove the target values from the dataset
X_train_df = X_train_df.drop(["Theta1", "Theta2", "Theta3"], axis=1)
X_test_df = X_test_df.drop(["Theta1", "Theta2", "Theta3"], axis=1)

# Normalizing the dataset
X_train_norm = (X_train_df - X_train_df.mean()) / X_train_df.std()
X_test_norm = (X_test_df - X_test_df.mean()) / X_test_df.std()

# Convert to numpy arrays
X_train = X_train_norm.values
X_valid = X_test_norm.values

y1_train = y1_train.reshape(-1, 1) # Reshape y1_train to match X_train
y2_train = y2_train.reshape(-1, 1) # Reshape y2_train to match X_train
y3_train = y3_train.reshape(-1, 1) # Reshape y3_train to match X_train

y1_test = y1_test.reshape(-1, 1) # Reshape y1_train to match X_train
y2_test = y2_test.reshape(-1, 1) # Reshape y2_train to match X_train
y3_test = y3_test.reshape(-1, 1) # Reshape y2_train to match X_train

y_train = (y1_train, y2_train, y3_train)
y_valid = (y1_test, y2_test, y3_test)


From local


<h2 style="font-size: 20px;">CREATING THE MODEL</h2>


In [2]:
def train_model(X_train, y_train, X_valid, y_valid, input_nodes, hidden_nodes, hidden_nodes_2, output_nodes, learning_rate, batch_size, epochs, dropout_rate):
    """
    Train a neural network model for multi-output regression.

    Args:
    X_train (numpy.ndarray): Input features for training.
    y_train (tuple of numpy.ndarray): Target labels for training.
    X_valid (numpy.ndarray): Input features for validation.
    y_valid (tuple of numpy.ndarray): Target labels for validation.
    input_nodes (int): Number of nodes in the input layers of the neural network.
    hidden_nodes (int): Number of nodes in the first hidden layer of the neural network.
    hidden_nodes_2 (int): Number of nodes in the second hidden layer of the neural network.
    output_nodes (tuple of int): Number of nodes in the output layers of the neural network.
    learning_rate (float): Learning rate for the optimizer.
    batch_size (int): Number of samples per batch during training.
    epochs (int): Number of epochs for training.
    dropout_rate (float): Dropout rate for regularization.

    Returns:
    tuple: A tuple containing:
        - nn_model (keras.Model): Trained neural network model.
        - history (dict): Training history.
    """
    # Unpack target labels
    y_train
    y_valid

    # Define input layer
    input_layer = Input(shape=(input_nodes,))

    # Define dense layers
    dense_layer_1 = Dense(units=hidden_nodes, activation="relu")(input_layer)
    dropout_layer_1 = Dropout(dropout_rate)(dense_layer_1)  # Adding dropout after the first dense layer

    
    dense_layer_2 = Dense(units=hidden_nodes, activation="relu")(dropout_layer_1)
    dropout_layer_2 = Dropout(dropout_rate)(dense_layer_2)  # Adding dropout after the second dense layer

    dense_layer_3 = Dense(units=hidden_nodes_2, activation="relu")(dropout_layer_2)
    dropout_layer_3 = Dropout(dropout_rate)(dense_layer_3)
    
    # Define Y1 output
    y_output = Dense(units=1, activation="linear", name="y_output")(dropout_layer_3)

    

    # Define the model with the input layer and a list of outputs
    model = Model(inputs=input_layer, outputs=y_output)

    # Specify the optimizer
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss={'y_output': 'mse'},
                  metrics={'y_output': tf.keras.metrics.RootMeanSquaredError()})

    # Train the model
    history = model.fit(X_train, {'y_output': y_train},
                        validation_data=(X_valid, {'y_output': y_valid}),
                        epochs=epochs, batch_size=batch_size, verbose=2)

    return model, history

import matplotlib.pyplot as plt

def plot_history(history):
    """
    Show the difference in loss and RMSE with each epoch of training the model.

    Args:
       history : Keras History object that stores information about the trained model.

    Returns:
        Plot showing the loss and RMSE with each epoch.
    """
    fig, axs = plt.subplots(1, 2, figsize=(20, 8))

    # Plot loss
    axs[0].plot(history.history['loss'], label='loss')
    axs[0].plot(history.history['val_loss'], label='val_loss')
    axs[0].set_xlabel('Epoch')
    axs[0].set_ylabel('Loss')
    axs[0].grid(True)
    axs[0].legend()

    # Plot RMSE
    axs[1].plot(history.history['root_mean_squared_error'], label='RMSE')
    axs[1].plot(history.history['val_root_mean_squared_error'], label='val_RMSE')
    axs[1].set_xlabel('Epoch')
    axs[1].set_ylabel('RMSE')
    axs[1].grid(True)
    axs[1].legend()

    plt.show()


<h2 style="font-size: 20px;">PARAMETERS OF THE MODEL</h2>

In [37]:
input_nodes = len(X_train[1])
output_nodes = 1
epochs = 200 #100 było git
hidden_nodes = 128  #60 było git
hidden_nodes_2 = 64  #64 było git
dropout_rate = 0.4
#0.25 było git
batch_size = 1000 #wczesniej 1000
learining_rate = 0.000001 #0.0001 było git

<h2 style="font-size: 20px;">TESTETING THE MODEL</h2>

In [6]:
model_y1, history_y1 = train_model(X_train, y_train[0], X_valid, y_valid[0], input_nodes, hidden_nodes, hidden_nodes_2, output_nodes, learining_rate, batch_size, epochs, dropout_rate)

Epoch 1/150
11664/11664 - 19s - 2ms/step - loss: 1537.3088 - root_mean_squared_error: 39.2085 - val_loss: 582.4927 - val_root_mean_squared_error: 24.1349
Epoch 2/150
11664/11664 - 22s - 2ms/step - loss: 588.5819 - root_mean_squared_error: 24.2607 - val_loss: 328.3848 - val_root_mean_squared_error: 18.1214
Epoch 3/150
11664/11664 - 20s - 2ms/step - loss: 377.7649 - root_mean_squared_error: 19.4362 - val_loss: 226.6269 - val_root_mean_squared_error: 15.0541
Epoch 4/150
11664/11664 - 17s - 1ms/step - loss: 281.6157 - root_mean_squared_error: 16.7814 - val_loss: 172.8768 - val_root_mean_squared_error: 13.1483
Epoch 5/150
11664/11664 - 17s - 1ms/step - loss: 227.1324 - root_mean_squared_error: 15.0709 - val_loss: 142.1157 - val_root_mean_squared_error: 11.9212
Epoch 6/150
11664/11664 - 18s - 2ms/step - loss: 190.7086 - root_mean_squared_error: 13.8097 - val_loss: 117.5524 - val_root_mean_squared_error: 10.8422
Epoch 7/150
11664/11664 - 18s - 2ms/step - loss: 165.0330 - root_mean_squared_err

In [38]:
model_y2, history_y2 = train_model(X_train, y_train[1], X_valid, y_valid[1], input_nodes, hidden_nodes, hidden_nodes_2, output_nodes, learining_rate, batch_size, epochs, dropout_rate)

Epoch 1/200
4666/4666 - 16s - 3ms/step - loss: 7790.6968 - root_mean_squared_error: 88.2682 - val_loss: 776.6512 - val_root_mean_squared_error: 27.8676
Epoch 2/200
4666/4666 - 16s - 3ms/step - loss: 959.0604 - root_mean_squared_error: 30.9688 - val_loss: 554.5768 - val_root_mean_squared_error: 23.5483
Epoch 3/200
4666/4666 - 16s - 3ms/step - loss: 798.6995 - root_mean_squared_error: 28.2613 - val_loss: 479.2846 - val_root_mean_squared_error: 21.8914
Epoch 4/200
4666/4666 - 15s - 3ms/step - loss: 726.9827 - root_mean_squared_error: 26.9627 - val_loss: 443.7283 - val_root_mean_squared_error: 21.0636
Epoch 5/200
4666/4666 - 16s - 3ms/step - loss: 684.4787 - root_mean_squared_error: 26.1626 - val_loss: 422.9487 - val_root_mean_squared_error: 20.5644
Epoch 6/200
4666/4666 - 15s - 3ms/step - loss: 655.4660 - root_mean_squared_error: 25.6020 - val_loss: 407.6577 - val_root_mean_squared_error: 20.1893
Epoch 7/200
4666/4666 - 14s - 3ms/step - loss: 632.1976 - root_mean_squared_error: 25.1436 - 

In [44]:
model_y3, history_y3 = train_model(X_train, y_train[2], X_valid, y_valid[2], input_nodes, hidden_nodes, hidden_nodes_2, output_nodes, learining_rate, batch_size, epochs, dropout_rate)

Epoch 1/200
4666/4666 - 15s - 3ms/step - loss: 10406.6826 - root_mean_squared_error: 102.0137 - val_loss: 9512.3086 - val_root_mean_squared_error: 97.5300
Epoch 2/200
4666/4666 - 14s - 3ms/step - loss: 3640.4158 - root_mean_squared_error: 60.3372 - val_loss: 1569.8383 - val_root_mean_squared_error: 39.6210
Epoch 3/200
4666/4666 - 13s - 3ms/step - loss: 1509.8561 - root_mean_squared_error: 38.8573 - val_loss: 716.0472 - val_root_mean_squared_error: 26.7590
Epoch 4/200
4666/4666 - 14s - 3ms/step - loss: 851.0077 - root_mean_squared_error: 29.1723 - val_loss: 317.0242 - val_root_mean_squared_error: 17.8053
Epoch 5/200
4666/4666 - 14s - 3ms/step - loss: 658.4291 - root_mean_squared_error: 25.6600 - val_loss: 241.3067 - val_root_mean_squared_error: 15.5342
Epoch 6/200
4666/4666 - 15s - 3ms/step - loss: 566.0393 - root_mean_squared_error: 23.7916 - val_loss: 190.0648 - val_root_mean_squared_error: 13.7865
Epoch 7/200
4666/4666 - 15s - 3ms/step - loss: 501.8645 - root_mean_squared_error: 22.4

In [36]:
plot_history(history_y1)

Y1_pred = model_y1.predict(X_valid)

36450/36450 ━━━━━━━━━━━━━━━━━━━━ 24s 662us/step


In [39]:
plot_history(history_y2)

Y2_pred = model_y2.predict(X_valid)

36450/36450 ━━━━━━━━━━━━━━━━━━━━ 22s 615us/step


In [16]:
plot_history(history_y3)

Y3_pred = model_y3.predict(X_valid)

36450/36450 ━━━━━━━━━━━━━━━━━━━━ 22s 588us/step


In [87]:
plt.scatter(y1_test, Y1_pred)
combined_array1 = np.column_stack((y1_test, Y1_pred))

In [86]:
combined_array1

plt.scatter(y2_test, Y2_pred)
combined_array2 = np.column_stack((y2_test, Y2_pred))


ValueError: x and y must be the same size

In [62]:
combined_array2

plt.scatter(y3_test, Y3_pred)
combined_array3 = np.column_stack((y3_test, Y3_pred))

In [24]:
def scale_this_point(point_in_space):
    X , Y , Z =   point_in_space
    X_norm = (X - X_train_df.loc[:,'X'].mean()) / X_train_df.loc[:,'X'].std()
    Y_norm = (Y - X_train_df.loc[:,'Y'].mean())/ X_train_df.loc[:,'Y'].std()
    Z_norm = (Z - X_train_df.loc[:,'Z'].mean()) / X_train_df.loc[:,'Z'].std()
    
    return X_norm, Y_norm, Z_norm

point = np.array([-6.123234e-16, -5.000000e+00,  9.000000e+00])

point_in_space_norm = scale_this_point(point)
print(point_in_space_norm)
point_in_space_norm[0]

#prediction on the point
NEW_point_to_predict = np.array([[point_in_space_norm[0], point_in_space_norm[1],point_in_space_norm[2]]])
Theta1 = model_y1.predict(NEW_point_to_predict)
Theta2 = model_y2.predict(NEW_point_to_predict)
Theta3 = model_y3.predict(NEW_point_to_predict)

(0.0031068498609969223, -1.1260752622503893, 0.46325391521788817)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [25]:
print(Theta1)
print(Theta2)
print(Theta3)

[[90.039]]
[[84.54559]]
[[89.46773]]


In [40]:
import os

# Define the directory path
directory = r'C:\Users\1mich\PYTHON\MODEL'


# Create the directory if it doesn't exist
os.makedirs(directory, exist_ok=True)

In [47]:
# Now save your model
file_path_y1 = os.path.join(directory, 'MODEL_50_1_ONE_TARGET_AT_TIME_y1.keras')

# Now save your model
model_y1.save(file_path_y1)

In [46]:
model_y2.save(os.path.join(directory, 'MODEL_50_1_ONE_TARGET_AT_TIME_y2.keras'))


In [45]:
model_y3.save(os.path.join(directory, 'MODEL_50_1_ONE_TARGET_AT_TIME_y3.keras'))